# 🦸🏻 Comprendiendo el Semantic Kernel con Héroes 🦸🏼‍♀️

Semantic Kernel ha sido creado para permitir a los desarrolladores integrar Inteligencia Artificial en sus aplicaciones de manera fluida. Para lograr esto, proporciona un conjunto de características que te permitirán añadir modelos, prompts, funciones nativas y memorias sin requerir un conocimiento profundo de IA 🥲. Por eso se dice que Semantic Kernel simula el cerebro 🧠 de tu aplicación.

<div style="text-align:center">
    <img src="images/semantic-kernel.png" width="15%">
</div>

## (Opcional) - Creando un Servicio Azure Open AI 🤖 y Despliegues

Antes de sumergirnos en Semantic Kernel 🛝, necesitarás tener configurado uno de los servicios soportados. Actualmente, puedes elegir entre las siguientes opciones: [Azure Open AI](https://azure.microsoft.com/en-us/products/ai-services/openai-service), [Open AI](https://openai.com/), o [Hugging Face](https://huggingface.co/).

En este ejemplo, voy a usar Azure Open AI.

Por lo tanto, a través de Azure CLI, necesito iniciar sesión:


In [ ]:
az login

Si lo necesitas, porque tienes muchas suscripciones en tu cuenta, puedes seleccionar la que te interesa (o la que tiene acceso a Azure Open AI 😊) a través del siguiente comando:


In [ ]:
az account set -n "Visual Studio Enterprise Subscription"

Ahora, para poder crear lo que necesitas para este cuaderno, establece las siguientes variables con tus valores preferidos:


In [ ]:
$RESOURCE_GROUP="understanding-semantic-kernel"
$LOCATION="canadaeast"
$AZURE_OPEN_AI="ai-for-heroes"

Con ellos, ahora puedes crear el grupo de recursos:


In [ ]:
az group create `
--name $RESOURCE_GROUP `
--location $LOCATION

Un recurso de Azure Open AI:


In [ ]:
az cognitiveservices account create `
--name $AZURE_OPEN_AI `
--custom-domain $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--kind OpenAI `
--sku S0 `
--location $LOCATION

Y finalmente, necesitas un despliegue de cualquiera de los modelos que tienes disponibles. En este ejemplo, voy a usar gpt-4:

In [ ]:
az cognitiveservices account deployment create `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--deployment-name "gpt-4" `
--model-name "gpt-4" `
--model-version "0613"  `
--model-format OpenAI `
--sku-capacity "10" `
--sku-name "Standard"

El parámetro **sku-capacity** nos permite especificar cuántos tokens por minuto podemos enviar a este modelo. Para ver cómo se está utilizando tu cuota, puedes usar este otro comando:


In [ ]:
az cognitiveservices usage list `
-l $LOCATION

Carga las variables de entorno con tu endpoint y clave de Azure Open AI:

In [ ]:
$env:AZURE_OPEN_AI_KEY =$(az cognitiveservices account keys list `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "key1" `
--output tsv)

$env:AZURE_OPEN_AI_ENDPOINT =$(az cognitiveservices account show `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "properties.endpoint" `
--output tsv)

#dir env:AZURE_OPEN_AI_KEY
#dir env:AZURE_OPEN_AI_ENDPOINT

## Cómo empezar con Semantic Kernel


Lo primero que necesitas para poder ejecutar Semantic Kernel en este cuaderno es instalar la biblioteca **Microsoft.SemanticKernel**, que actualmente está en la versión **1.5.0** (prometo seguir actualizando 🤓).

Puedes encontrar más información sobre esta biblioteca y sus actualizaciones en la página oficial de NuGet [aquí](https://www.nuget.org/packages/Microsoft.SemanticKernel).


In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"

Installed Packages Microsoft.SemanticKernel, 1.5.0

Con ello, ahora puedes instanciar el cerebro de tu aplicación a través de **KernelBuilder**. Tiene muchos conectores a los modelos y otras cosas, pero por ahora, comencemos con lo básico:


In [2]:
using Microsoft.SemanticKernel;

// Create a Builder
var builder = Kernel.CreateBuilder();

Dependiendo del tipo de tarea que quieras realizar, puedes usar diferentes métodos con el prefijo **With**. Por ahora, vamos a usar algunos de los modelos de tipo **completion** o completado.


In [3]:
// OpenAI keys
var modelId = "gpt-4";
var apiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key")).GetClearTextPassword();

// Create a chat completion service
builder.AddOpenAIChatCompletion(modelId, apiKey);

In [ ]:
// Azure OpenAI keys
var deploymentName = "gpt-4";
var endpoint = Environment.GetEnvironmentVariable("AZURE_OPEN_AI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPEN_AI_KEY");

// Create a chat completion service
builder.AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey);

Con la configuración realizada, lo único que queda es generar el kernel con todo configurado:


In [11]:
var kernel = builder.Build();

## Uso de plugins de Semantic Kernel

Los plugins son el núcleo de Semantic Kernel. Con ellos, encapsulas capacidades para que sean reutilizables, mantenibles y planificables (lo entenderás más tarde 🙃). Hay dos tipos: aquellos que consisten en plantillas de indicaciones llamadas **Semantic Functions** y funciones nativas del lenguaje de programación elegido llamadas **Native Functions**.

<div style="text-align:center">
    <img src="images/writer-plugin-example.png" width="40%" />
</div>

## Funciones Semánticas

Cuando hablas con modelos de inteligencia artificial, debes hacerlo con lo que se conoce como un *prompt*. Esto puede variar desde una simple frase hasta algo más elaborado, permitiendo que el modelo comprenda no solo lo que queremos sino también cómo lo queremos. Si echas un vistazo a la documentación oficial, define este tipo de función como la boca 👄 y los oídos 👂🏻 de tu cerebro 🧠. 🤖🌐

<div style="text-align:center">
    <img src="images/semantic-function-explainer.png" width="20%" />
</div>

Como parte de este repositorio, tienes una carpeta llamada **SemanticFunctions** que tiene diferentes funciones de este tipo:

- **FunPlugin**: Esto nos permite pedirle al modelo que haga bromas sobre héroes bajo ciertas condiciones, a través de la función **Joke**.
- **WritePlugin**: Para mostrarte que dentro de un plugin puedes tener diferentes funciones, en este directorio tenemos dos relacionadas con el arte de escribir: la primera, **OOF**, nos permite generar el mensaje "Fuera de la oficina" 🏢📧 para superhéroes y la segunda **StoryGen** nos ayudará a crear historias, también sobre superhéroes 🦸🏻‍♂️🦸🏻‍♀️.

Para que nuestro kernel sepa que estos plugins están disponibles, primero necesitas obtener la ruta del directorio:


In [12]:
using System.IO;

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "FunPlugIn");

Y luego comienza a cargar los plugins que deseas:


In [13]:
// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectory);

Para poder invocar una función de este tipo, puedes hacerlo de la siguiente manera:


In [14]:
var joke = new KernelArguments() { ["input"] = "cuentame un chiste" };
var result = await kernel.InvokeAsync(funPluginFunctions["JokeSpa"], joke);

Console.WriteLine(result.GetValue<string>());

Claro, aquí tienes un chiste:

¿Por qué Superman nunca pelea contra un chiste malo?

Porque siempre se ríe hasta quedar exhausto.


Como puedes ver, solo usando **kernel.RunAsync** y pasando mi solicitud como parámetros y qué función, en este caso **JokeSpa**, dentro del plugin, en este caso **FunPlugin**, quiero usar.

Cada una de estas funciones consta de dos archivos:

- **skprompt.txt** es el archivo donde se define el prompt para enviar al modelo en un formato de plantilla, de modo que pueda recibir parámetros y hacerlos más reutilizables.


    ```
    WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE SUBJECT BELOW

    JOKE MUST BE:
    - IN SPANISH

    BE CREATIVE AND FUNNY. I WANT TO LAUGH.

    Incorporate the hero if provided: {{$hero}}
    +++++

    {{$input}}
    +++++
    ```

- **config.json**, which allows us to indicate the maximum number of tokens allowed for this call (**max_tokens**), the temperature to control the randomness of the responses (**temperature**), which means that closer to 1 they will be more random and closer to zero more determined and focused on the most likely response, **top_p** is used to control the diversity of the responses, where a value of 0.0 means that only the most likely responses will be considered and 1 where all possible responses will be considered and **presence_penalty** and **frequency_penalty** to adjust the penalty for the presence and frequency of the tokens in the generated responses. On the other hand, if the skprompt.txt file receives parameters, they must also be defined in this file in the **parameters** array of the **input** object.

  ```javascript
  {
    "schema": 1,
    "description": "Generate a funny joke about heroes",
    "models": [
      {
        "max_tokens": 150,
        "temperature": 0.9,
        "top_p": 0.5,
        "presence_penalty": 0.2,
        "frequency_penalty": 0.3
      }
    ],
    "input": {
      "parameters": [
        {
          "name": "input",
          "description": "Joke subject",
          "defaultValue": ""
        },
        {
          "name": "hero",
          "description": "Give a hint about the hero you want to joke about",
          "defaultValue": ""
        }
      ]
    }
  }
  ```


Como podemos ver, en esta ejecución, solo hemos pasado el primer parámetro: INPUT, pero no hemos pasado el parámetro HERO. No hemos especificado el héroe que queremos utilizar para generar el chiste. Para poder enviar más de un parámetro necesitas crear un objeto del tipo **ContextVariables**:

In [15]:
var variables = new KernelArguments() {
    ["input"] = "Cuentame un chiste de futbol",
    ["hero"] = "Ironman"
};

Para poder usar estos dos valores como parte de la llamada, simplemente colócalos como el primer argumento.


In [16]:
var result = await kernel.InvokeAsync(funPluginFunctions["JokeSpa"], variables);

Ahora verifica si el chiste es sobre el héroe especificado como parámetro.


In [17]:
Console.WriteLine(result.GetValue<string>());

¿Por qué Ironman no puede jugar al fútbol?

Porque cada vez que recibe el balón, se convierte en un hombre de hierro gol.


De la misma manera, podemos usar las funciones incluidas en **WriterPlugin**:


In [18]:
// Load the WriterPlugin from the Plugins Directory
var pluginsDirectoryWriter = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "WriterPlugin");

var writerPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectoryWriter);


Una que genera historias:


In [19]:
var variables = new KernelArguments() {
    ["input"] = "Cuenta una historia de tennis",
    ["hero"] = "Ironman"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["StoryGenSpa"], variables);
Console.WriteLine(result);

Había una vez un héroe llamado Ironman 🦾💪🚀, que era conocido por sus increíbles habilidades de combate y su inteligencia. Sin embargo, pocos sabían que Ironman también tenía una pasión secreta: jugar al tenis 🎾.

Un día ☀️, Ironman jugaba su partido habitual de tenis solo en un antiguo campo de juego 🏞️. De repente, apareció un formidable contrincante: un robot gigante 🤖 de uno de sus archienemigos. El robot había venido a desafiar a Ironman a un partido de tenis. Ironman, sorprendido pero listo para el desafío, aceptó y comenzaron el partido.

En el medio del juego 🎾🏆, Ironman luchaba para mantenerse al día. El robot gigante era muy rápido y preciso, y Ironman estaba perdiendo el control del juego. Pero recordó el espíritu de combate y la dedicación que siempre le habían llevado al éxito en sus batallas. Entonces, se puso su casco de Ironman y decidió darlo todo en la cancha.

Finalmente, con gran esfuerzo y coraje, Ironman logró remontar el juego. Con un último y poderoso saque ⚡ que

O incluso para crear mensajes para cuando los héroes 🦸🏻‍♀️ están de vacaciones ✈️🚢🌴:


In [20]:
var variables = new KernelArguments() {
    ["input"] = "Crea un OOF para Navidad",
    ["hero"] = "Hulk"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["OOFSpa"], variables);
Console.WriteLine(result.GetValue<string>());

Estimados colegas,

Como muchos de ustedes ya sabrán, tengo algunas habilidades poco comunes. Sí, soy Hulk 💪🟢, y justo ahora, necesito tomar unas vacaciones para mantener mi "enfado" bajo control.

Voy a estar fuera de la oficina desde el 24 de diciembre hasta el 5 de enero por las festividades de Navidad 🎄 y el Año Nuevo 🎆. Durante este tiempo, estaré en un lugar secreto, descansando y recargando mis súper fuerzas.

Durante mi ausencia, no tendrán que preocuparse. Mi querido amigo Thor ⚡🔨 se quedará a cargo. Con su martillo Mjölnir y su valentía asgardiana, estoy seguro de que será capaz de resolver cualquier problema que pueda surgir.

Para cualquier consulta o emergencia, por favor, contacten con Thor en thor@asgard.com. Él estará más que feliz de ayudarles.

Finalmente, quiero aprovechar esta oportunidad para desearles a todos una Feliz Navidad 🎅🎁y un próspero Año Nuevo lleno de amor, paz y felicidad.

Nos vemos en enero, ¡y recuerden, siempre mantengan la calma! 😉

Saludos,

Hulk


## Funciones Nativas

Mientras que las funciones semánticas nos permiten definir y reutilizar prompts, **con las funciones nativas puedes hacer que el kernel semántico llame a funciones escritas en C# o Python**, para tareas que van más allá de una llamada a través de un prompt.

<div style="text-align:center">
    <img src="images/native-function-explainer.png" width="25%">
</div>

### ¿Por qué necesito funciones nativas en este tipo de aplicaciones?

Los modelos de lenguaje grandes (LLMs) son excelentes para generar texto, pero hay varias tareas que no pueden realizar por sí mismos. Estos incluyen, entre otros:

- Recuperar datos de fuentes de datos externas
- Saber qué hora es
- Realizar operaciones matemáticas complejas
- Completar tareas en el mundo real
- Memorizar y recordar información

Para estos escenarios, y muchos otros, las funciones nativas son muy útiles 👍🏻


Para este ejemplo, voy a utilizar una API llamada **SuperHero API**, que requiere una clave de API. Puedes obtenerla de su sitio web: [https://superheroapi.com/](https://superheroapi.com/)

Una vez que la tengas, pásala al prompt que aparece con la siguiente línea:


In [21]:
var superHeroApiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Super Hero Api key")).GetClearTextPassword();

Ahora, para cargar una función nativa, debemos agregar esto antes de construir el Kernel.

Vamos a crear una nueva clase Super Hero Info y agregarla como un Plugin.


In [22]:
#load "NativeFunctions/HeroInfo.cs"

var heroInfo = new HeroInfo(superHeroApiKey);
builder.Plugins.AddFromObject(heroInfo, "HeroInfo");
Kernel kernel = builder.Build();

El constructor ahora puede interactuar con la función local. Hagamos una llamada para obtener el alter ego de un superhéroe.
The builder now can interact with the local function. Let's make a call to get the alter ego of a super hero.

In [23]:
var kernelArgs = new KernelArguments()
{
    ["input"] = "Ironman"
};

var result = await kernel.InvokeAsync<string>("HeroInfo", "GetAlterEgo", kernelArgs);
Console.WriteLine(result);

Tony Stark


En este repositorio, hay otro directorio llamado **NativeFunctions** donde puedes encontrar una clase llamada **HeroInfo.cs**. En ella, hay una función decorada con el atributo **KernelFunction**, que nos permite indicar al Kernel Semántico que es una función nativa y, a través de la propiedad **Description**, también darle información sobre cuál es el propósito de esta función. En este caso, lo que este método nos permite hacer es recuperar el alter ego del superhéroe que pasamos como parámetro, en este ejemplo el de Ironman.


## Planner (Planificador)

Hasta ahora, todos los complementos que has visto se han ejecutado intencionalmente. Es decir, nadie los ha elegido por ti y puedes ejecutarlos según tus necesidades. Sin embargo, esta es la forma de interactuar con **Semantic Kernel** más *estática*. Existe otra opción llamada **Planificador** que te dejará asombrado 😮

El Planificador es una función que toma la solicitud del usuario y devuelve un plan sobre cómo llevar a cabo dicha solicitud. Para hacerlo, utiliza la inteligencia artificial para combinar los complementos registrados en el núcleo y recombinarlos en una serie de pasos que completan un objetivo.

<div style="text-align:center;">
    <img src="images/the-planner.png" width=35% />
</div>

Para verlo en acción, vamos a utilizar los complementos que ya conoces.

Lo primero que necesitas es agregar la referencia nuget al **Planner Handlebars**.

Para aprender más sobre este paquete NuGet, puedes visitar [aquí](https://www.nuget.org/packages/Microsoft.SemanticKernel.Planners.Handlebars/1.5.0-preview#show-readme-container).


In [24]:
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.5.0-preview"

Installed Packages Microsoft.SemanticKernel.Planners.Handlebars, 1.5.0-preview

Ahora estamos listos para crear un planificador.


In [25]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner();


Existen diferentes tipos como puedes ver [aquí](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/planners/?tabs=Csharp). En este ejemplo, vamos a utilizar el que ejecuta tareas de forma secuencial.

Dado que ya tienes todos los complementos cargados en tu instancia de kernel, podemos preguntarle algo como esto:


In [26]:
using System.Text.Json;

var ask = "Me gustaría que me cuentes un chiste sobre Batman, y con ese chiste, crea un mensaje de fuera de la oficina utilizando la broma.";
#pragma warning disable SKEXP0060
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 0: Set the joke about Batman --}}
{{set "joke" "¿Por qué Batman puso a su coche de remate? ¡Para tener un bat-móvil!"}}

{{!-- Step 1: Set the out of office message template --}}
{{set "officeMessageTemplate" "Estoy fuera de la oficina en este momento. Mientras tanto, aquí tienes un chiste para animarte el día: \n\n"}}

{{!-- Step 2: Concatenate the message template and the joke --}}
{{set "outOfOfficeMessage" (concat officeMessageTemplate joke)}}

{{!-- Step 3: Print the final out of office message --}}
{{json outOfOfficeMessage}}


Como puedes ver, el planificador conoce los complementos gracias a la descripción que incluí como parte de su implementación.

Es hora de ejecutar el plan y obtener un mensaje de fuera de la oficina para **Batman** que incluye un **chiste malo**.


In [27]:
// executing the plan
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(originalPlanResult.ToString());

Original Plan results:

Estoy fuera de la oficina en este momento. Mientras tanto, aquí tienes un chiste para animarte el día: \n\n¿Por qué Batman puso a su coche de remate? ¡Para tener un bat-móvil!


### Ejemplos avanzados con Planner

El kernel ya tiene los plugins cargados, y la función, veamos si puede resolver un ejemplo más complejo:


In [ ]:
var ask_complex = @"Me gustaría que averigües el alter ego de IronMan y 
crees un mensaje de fuera de la oficina para IronMan, firmando con su alter ego.";

#pragma warning disable SKEXP0060
var plan_complex = await planner.CreatePlanAsync(kernel, ask_complex);

Console.WriteLine("Plan:\n");
Console.WriteLine(plan_complex);

Ejecutemos el plan y veamos su salida:


In [ ]:
#pragma warning disable SKEXP0060
var complexPlanResult = await plan_complex.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Complex  Plan results:\n");
Console.WriteLine(complexPlanResult.ToString());

## Memoria del Kernel (Kernel Memory)

<img src="images/How kernel memory works.png" width="80%" />

Para poder utilizar la Memoria del Kernel, necesitas agregar su biblioteca nuget, además de importar la clase que he generado en el directorio **KernelMemory**.

Para aprender más sobre la biblioteca de Memoria del Kernel, puedes visitar la [página del paquete NuGet](https://www.nuget.org/packages/Microsoft.SemanticKernel.Plugins.Memory/1.5.0-alpha#show-readme-container).


In [ ]:
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.5.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"


Para utilizar la memoria, necesitamos instanciar el Plugin de Memoria con un Almacenamiento de Memoria y un backend de incrustación. En este ejemplo, hacemos uso de VolatileMemoryStore que puede considerarse como un almacenamiento temporal en memoria (no confundir con la Memoria Semántica).

Esta memoria no se escribe en el disco y solo está disponible durante la sesión de la aplicación.

Al desarrollar tu aplicación tendrás la opción de conectar un almacenamiento persistente como Azure Cosmos Db, PostgreSQL, SQLite, etc. La Memoria Semántica también permite indexar fuentes de datos externas, sin duplicar toda la información, más sobre eso más adelante.


In [ ]:
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// create the memory builder
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0052
var memoryBuilder = new MemoryBuilder();

// Using OpenAI
memoryBuilder.WithOpenAITextEmbeddingGeneration("text-embedding-ada-002", apiKey);

// Using Azure OpenAI 
// memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(
//         AzureOpenAI.EmbeddingsModel,
//          AzureOpenAI.Endpoint,
//         AzureOpenAI.ApiKey,
//         "text-embedding-ada-002");

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());
var memory = memoryBuilder.Build();

Creemos algunas memorias iniciales "Fan Facts". Podemos agregar memorias a nuestro VolatileMemoryStore utilizando SaveInformationAsync:


In [ ]:
// add fan facts to the collection
const string MemoryCollectionName = "fanFacts";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "Gisela's favourite super hero is Batman");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "The last super hero movie watched by Gisela was Guardians of the Galaxy Vol 3");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "Bruno's favourite super hero is Invincible");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "The last super hero movie watched by Bruno was Aquaman II");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "Bruno don't like the super hero movie: Eternals");

Y ahora podemos probar y buscar en la memoria:

In [ ]:
var questions = new[]
{
    "what is Bruno's favourite super hero?",
    "what was the last movie watched by Gisela?",
    "Which is the prefered super hero for Gisela?",
    "Did Bruno watched a super hero movie in the past, which was the last one?"
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}
